In [6]:
from __future__ import print_function, division
import os
from PIL import Image
import pandas as pd


import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.autograd import Variable
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy

plt.ion()

use_gpu = torch.cuda.is_available()
if use_gpu:
    print("Using CUDA")

In [37]:
from torchvision import transforms
transform = transforms.Compose([            #[1]
 transforms.Resize(256),                    #[2]
 transforms.CenterCrop(224),                #[3]
 transforms.ToTensor(),                     #[4]
 transforms.Normalize(                      #[5]
 mean=[0.485, 0.456, 0.406],                #[6]
 std=[0.229, 0.224, 0.225]                  #[7]
 )])

vgg16 = models.vgg16_bn()
for param in vgg16.features.parameters():
    param.require_grad = False
class_names = ['cloud', 'other', 'smoke']

num_features = vgg16.classifier[6].in_features
features = list(vgg16.classifier.children())[:-1] # Remove last layer
features.extend([nn.Linear(num_features, len(class_names))]) # Add our layer with 4 outputs
vgg16.classifier = nn.Sequential(*features) # Replace the model classifier

vgg16.load_state_dict(torch.load('/home/jszumny/Documents/copy of smoke data/newnew/attempts/3features/fullmodelbest.pt',map_location=torch.device('cpu')))

resnet18 = torch.hub.load('pytorch/vision:v0.10.0', 'resnet18', pretrained=True)
num_features = resnet18.fc.in_features     #extract fc layers features
resnet18.fc = nn.Linear(num_features, 3) #(num_of_class == 2)


Using cache found in /home/jszumny/.cache/torch/hub/pytorch_vision_v0.10.0
/home/jszumny/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/jszumny/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [39]:
dir = "/home/jszumny/Arm Data/"

data = []
for file in os.listdir(dir):
    
    for image in os.listdir(dir +file):

        fullimage = Image.open(dir + "/" +  file + "/" +image)
        fullimage = fullimage.resize((1344, 1120))

        for i in range(6):
            for k in range(5):
                
                tile = fullimage.crop((i*224, k*224, (i+1)*224, (k+1)*224))
                
                resnet18.load_state_dict(torch.load('/home/jszumny/Documents/copy of smoke data/newnew/attempts/3features/resnet18model.pt',map_location=torch.device('cpu')))

                img_t = transform(tile)
                batch_t = torch.unsqueeze(img_t, 0)

                resnet18.eval()
                out = resnet18(batch_t)
            
                _, index = torch.max(out, 1)
            
                percentage = torch.nn.functional.softmax(out, dim=1)[0] * 100
                
                d = {"filepath": dir+file+image, "tilenum": str(i) + str(k),"class": class_names[index[0]], "percentage": percentage[index[0]].item()}
                data.append(d)

KeyboardInterrupt: 

In [64]:
df = pd.DataFrame(data)
smoke = df[df['class']=='smoke']
smoke



,filepath,tilenum,class,percentage
35,/home/jszumny/Arm Data/enatwrcamtarmacC1.a1.20...,10,smoke,80.810905
40,/home/jszumny/Arm Data/enatwrcamtarmacC1.a1.20...,20,smoke,52.948685
95,/home/jszumny/Arm Data/enatwrcamtarmacC1.a1.20...,10,smoke,57.915951
100,/home/jszumny/Arm Data/enatwrcamtarmacC1.a1.20...,20,smoke,51.488560
260,/home/jszumny/Arm Data/enatwrcamtarmacC1.a1.20...,40,smoke,63.598591
266,/home/jszumny/Arm Data/enatwrcamtarmacC1.a1.20...,51,smoke,44.133846
275,/home/jszumny/Arm Data/enatwrcamtarmacC1.a1.20...,10,smoke,52.734058
285,/home/jszumny/Arm Data/enatwrcamtarmacC1.a1.20...,30,smoke,93.115051
305,/home/jszumny/Arm Data/enatwrcamtarmacC1.a1.20...,10,smoke,52.538776
335,/home/jszumny/Arm Data/enatwrcamtarmacC1.a1.20...,10,smoke,63.910538


In [41]:
len(df[df["class"] == 'smoke'])/len(df)

0.031026252983293555

In [31]:
df[df['class']=='smoke']

,filepath,tilenum,class,percentage
0,/home/jszumny/Arm Data/enatwrcamtarmacC1.a1.20...,00,smoke,70.337746
5,/home/jszumny/Arm Data/enatwrcamtarmacC1.a1.20...,10,smoke,89.789162
6,/home/jszumny/Arm Data/enatwrcamtarmacC1.a1.20...,11,smoke,98.951363
10,/home/jszumny/Arm Data/enatwrcamtarmacC1.a1.20...,20,smoke,65.970024
11,/home/jszumny/Arm Data/enatwrcamtarmacC1.a1.20...,21,smoke,77.337868
...,...,...,...,...
1211,/home/jszumny/Arm Data/enatwrcamtarmacC1.a1.20...,21,smoke,60.195953
1215,/home/jszumny/Arm Data/enatwrcamtarmacC1.a1.20...,30,smoke,54.713535
1220,/home/jszumny/Arm Data/enatwrcamtarmacC1.a1.20...,40,smoke,66.217506
1221,/home/jszumny/Arm Data/enatwrcamtarmacC1.a1.20...,41,smoke,80.955437
